# 01 - Simulating Free Particles in One Dimension

**Overview**: This notebook guides you through one of the simplest systems we can model, a collection of free classical particles (atoms, electrons, molecules) traveling in a one-dimensional simulation space, with hard-sphere interactions among themselves and with the walls of the simulation cell.  

In Python programs we will always rely on a variety of features that go beyond basic programming tools. These features are collected in specialized packages called Modules, which we need to import in our script/notebook in order to be able to use them. While we can import these modules at any time in our program, we will try to keep the declarations of modules at the very beginning of every notebook. 

In [ ]:
# @title Modules Setup
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# @title Simulation Parameters
velocity_scale = 10  # @param {type:"number"}
dt = 0.001  # @param {type:"number"}
nsteps = 3000  # @param {type:"integer"}
total_time = nsteps * dt

# Constants for simulation
box = 10.0

In [ ]:
# @title Run and Visualize the Simulation
# Initial conditions
position_a = 0.
velocity_a = 1.0 * velocity_scale
position_b = 5.
velocity_b = -2.0 * velocity_scale

trajectory_a = []
trajectory_b = []
time = []

t = 0
for _ in range(nsteps):
    time.append(t)
    trajectory_a.append(position_a)
    trajectory_b.append(position_b)

    position_a += velocity_a * dt
    position_b += velocity_b * dt

    # Elastic collision check
    if abs(position_b - position_a) < 1e-1:
        velocity_a, velocity_b = velocity_b, velocity_a

    # Reflect at walls
    if position_a > box or position_a < 0:
        velocity_a = -velocity_a
    if position_b > box or position_b < 0:
        velocity_b = -velocity_b

    t += dt

# --- Initial figure ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    row_heights=[0.4, 0.6],           # Bottom plot a bit taller
    vertical_spacing=0.20,            # Increase spacing between plots
    subplot_titles=("Particle Positions", "Trajectories")
)


# Initial positions
fig.add_trace(go.Scatter(x=[trajectory_a[0]], y=[0], mode='markers', marker=dict(size=15), name='A'), row=1, col=1)
fig.add_trace(go.Scatter(x=[trajectory_b[0]], y=[0], mode='markers', marker=dict(size=15), name='B'), row=1, col=1)

# Initial trajectory lines
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory_a[0]], mode='lines', name='A Trajectory',
                         line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory_b[0]], mode='lines', name='B Trajectory',
                         line=dict(color='red')), row=2, col=1)

# --- Animation frames ---
frames = []
frame_stride = 10  # Reduce total frames

for i in range(frame_stride, nsteps, frame_stride):
    frames.append(go.Frame(
        data=[
            go.Scatter(x=[trajectory_a[i]], y=[0], mode='markers', marker=dict(size=15)),
            go.Scatter(x=[trajectory_b[i]], y=[0], mode='markers', marker=dict(size=15)),
            go.Scatter(x=time[:i], y=trajectory_a[:i], mode='lines', line=dict(color='blue')),
            go.Scatter(x=time[:i], y=trajectory_b[:i], mode='lines', line=dict(color='red')),
        ]
    ))
fig.update_layout(
    height=600,
    title=f"1D Hard Sphere Simulation (velocity_scale={velocity_scale}, dt={dt})",
    
    # Top subplot: positions
    xaxis=dict(range=[0, box], title="Position"),
    yaxis=dict(showticklabels=False),
    
    # Bottom subplot: trajectories
    xaxis2=dict(title="Time", range=[0, nsteps * dt]),       # ✅ full time range
    yaxis2=dict(title="Position", range=[0, box]),           # ✅ full spatial range

    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {
                         "frame": {"duration": 30, "redraw": True},
                         "fromcurrent": True,
                         "transition": {"duration": 0}
                     }])
            ]
        )
    ]
)

# ✅ Don't forget to assign frames
fig.frames = frames
fig.show()